<div style="text-align: center;"> <h1>GROUP F MACHINE LEARNING FOR DATA SCIENCE FINAL PROJECT</h1><> </div>

This Jupyter Notebook will be the main program from our project. When the project is finished, we will be able to use this Jupyter code to generate the presentation slides.

In [3]:
import Scripts #Import all the scripts in the "Scripts" folder
import pandas as pd
import torch 
import numpy as np
import matplotlib as plt
import math
import sys
import sklearn
from sklearn.feature_selection import mutual_info_regression
import seaborn as sns



The first step to make is to import our training data and store it into a pandas structure and start the data preprocessing. Then we visualize it

In [4]:


dataPath = "./Data/TRAIN_Vehicles_Data.xlsx" #The route to the data
dataT = pd.read_excel(dataPath, na_values=["N", " ","Other","Nicht definiert"]) #After seeing the excel we set the null values as "N" and " "
#Save the output value
OutputTraining = dataT['LAID_UP_TIME']


plt.figure(figsize=(8, 6))
sns.histplot(dataT['LAID_UP_TIME'], kde=True, bins=30, color='blue')
plt.title("Distribution of LAID_UP_TIME", fontsize=16)
plt.xlabel("LAID_UP_TIME (days)")
plt.ylabel("Frequency")
plt.show()

plt.figure(figsize=(8, 6))
sns.boxplot(dataT['LAID_UP_TIME'], color='orange')
plt.title("Boxplot of LAID_UP_TIME", fontsize=16)
plt.xlabel("LAID_UP_TIME (days)")
plt.show()


TypeError: 'module' object is not callable

Step 1) By inspection, remove all the columns that we stimate that will be not useful.

In [15]:
dataT = dataT.drop(columns=["RPAKREP_VEHICLE_HKEY","CHASSIS_NUMBER","OFFICE_MAIN_BRAND","MANUFACTURER",
                            "VEHICLE_TYPE","VARIANT","RIM_KEY","UPHOLSTERY_CODE","ENGINE_ID","TRANSMISSION_TYPE",
                            "COMMISSION_NUMBER","LEASING_START","LEASING_END","LEASING_MILAGE","FINANCING_TYPE_NAME",
                            "PURCHASE_DATE","PRICE_LIST","SOLD_INVOICE_COSTUMER_ID","SOLD_CUSTOMER_ID","LAID_UP_TIME"])


NameError: name 'dataT' is not defined

Step 2) Eliminate the columns with more 10% of its values being null. 

In [16]:

null_percentage = dataT.isnull().mean() * 100 #Multiply the mean by 100 to pass to percentage
threshold = 10 #10% of null entries
columns_to_drop = null_percentage[null_percentage > threshold].index #Extract the index of the columns with more of 10% of its values being null
dataT = dataT.drop(columns=columns_to_drop) #Drop the columns
print("\nDataFrame after dropping columns with more than 10% null values:")
print(dataT)




DataFrame after dropping columns with more than 10% null values:
       COMPANY OFFICE MANUFACTURER_SHORT VEHICLE_GROUP  MILEAGE  \
0         44.0     TA                FOR           ECS   8600.0   
1         12.0     44                VOL           XC4      0.0   
2         27.0     33                FOR           TOC   1297.0   
3         33.0     K1                  V           POL   6020.0   
4         10.0     96                FOR           FOC      0.0   
...        ...    ...                ...           ...      ...   
99066     70.0      4                JAG           EPA     20.0   
99067     17.0     60                FOR           FIE  67748.0   
99068     33.0     KA                  V           AMA   3549.0   
99069     10.0     10                FOR           TNT  14117.0   
99070     17.0     61                ALF           STE  45013.0   

       OPERATING_HOURS  MILAGE_IN_FIELD  MILAGE_SALES  OPERATING_HOURS_SALES  \
0                  0.0              1.0        86

Step 3) Discard caterogic columns that increases the dimensionality in excess

In [17]:

categorical_cols = dataT.select_dtypes(include=['object', 'category']).columns
num_one_hot_encoding = dataT[categorical_cols].nunique()
excluded_cols = ['COLOR','VEHICLE_GROUP']
cols_to_remove = num_one_hot_encoding[(num_one_hot_encoding > 10) & 
         (~num_one_hot_encoding.index.isin(excluded_cols)) ].index.tolist()
print(f"Columns to drop (more than 15 categories ): {cols_to_remove}")
data_categoric_removed = dataT.drop(columns=cols_to_remove)
# Apply One Hot Encoding
data_encoded = pd.get_dummies(data_categoric_removed, drop_first=True)

Columns to drop (more than 15 categories ): ['OFFICE', 'MANUFACTURER_SHORT', 'UPHOLSTERY', 'TRANSMISSION_NAME', 'FUEL_TYPE_NAME', 'SOLD_INVOICE_COSTUMER_ID2', 'SALE_CUSTOMER_ID2']


In [ ]:


temp_df = data_encoded.copy()
temp_df["target"] = OutputTraining  
temp_df.dropna(inplace=True)

data_encoded_clean = temp_df.drop(columns=["target"])
OutputTraining_clean = temp_df["target"]

mi_scores = mutual_info_regression(
    data_encoded_clean,
    OutputTraining_clean,
    discrete_features=False
)

columns = data_encoded_clean.columns
threshold = 0.4

columns_to_drop = columns[mi_scores < threshold].tolist()
print("Columns to drop (MI < 0.4):", columns_to_drop)

data_cleaned = data_encoded_clean.drop(columns=columns_to_drop)

    
print(f"\nColumns to drop (Mutual Information < 0.4): {columns_to_drop}")
data_cleaned = data_encoded_clean.drop(columns=columns_to_drop)

KeyError: 'date_column'

Step 3) Statistic description of the data

In [56]:
n_columns = len(data_cleaned.columns)
print(f"After preprocesing we have: {n_columns} columns left ")
pd.set_option('display.max_columns', None) #To see all the columns
data_cleaned.describe()

After preprocesing we have: 13 columns left 


,SCALED_INVENTURAL_VALUE,SCALED_REPORT_VALUE
count,99004.000000,99004.000000
mean,0.013350,0.944832
std,0.031157,0.030980
min,0.000000,0.000000
25%,0.000000,0.950043
50%,0.000000,0.950043
75%,0.000000,0.950043
max,0.871569,0.963855


Step 4) Visualization of the variable type of the attributes

In [ ]:

data_cleaned.dtypes

for column in data_cleaned.columns:
    
    correlation = mutual_info_regression(data_cleaned[column],OutputTraining)
    
    
    print(f"Correlation between '{column}' y 'OutputTraining': {correlation:.4f}")

ModuleNotFoundError: No module named 'sklearn'